In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
url="https://www.arabam.com"
headersparam = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"}
urlMoney=[]
urls=[]
ozellikAd=[]

In [2]:
#Siteden gelen önemsiz verileri listeden siliyor
#Verilerin önündeki boşlukları kaldırıyor ve Km verisindeki noktayı kaldırıyor
def popItem(itemList,atName=False):
    dictItemList={}
    itemList.pop(0)
    itemList.pop(0)
    for x in range(len(itemList)):
        if atName:
            itemList[x]=strTrans(itemList[x].text).strip()
        else:
            dictItemList[strTrans(itemList[x].text).strip()]=strTrans(itemList[x].find_next_sibling().text).strip()
    if atName:
        return itemList
    return dictItemList

In [3]:
#Siteden gelen verilerin türkçe karakterlerini ingilizce yapıyor ve yazıyı küçük harfli haline çeviriyor
def strTrans(str):
    str=str.replace(':', '')
    s=str.maketrans("çğıöşüÇĞİÖŞÜ", "cgiosuCGIOSU")
    return str.translate(s).lower()

In [4]:
#Arama sonucunda ilk sonucun sitesine gidip sütünların adlarını aliyor
# ve ilerki veriler için dataframein boş bir şekilde iskeletini oluşturuyor
def dataFrameMake():
    query="/ikinci-el/otomobil/nissan"
    req = requests.get(url+query,headers=headersparam)
    soup = BeautifulSoup(req.text, "html.parser")
    c=soup.find("a", attrs={"class": "listing-text-new"}).get('href')
    req = requests.get(url+c,headers=headersparam)
    soup= BeautifulSoup(req.text, "html.parser")
    ozellikAd=popItem(soup.find("ul","detail-menu").findAll("span","bold"),True)
    df=pd.DataFrame(columns=ozellikAd)
    return df

In [5]:
#Siteden toplayacağı verilerin urllerini bulmak için siteyi geziyor ve bunları fiyatlarıyla birlikte bir diziye atıyor
#Ve fiyatları dataframe koyuyor böylece dataframede satırları yapmış oluyor 
import random
j=0
for i in range(1,11):
    query="/ikinci-el/otomobil/nissan?take=50&page="+str(i)
    req = requests.get(url+query,headers=headersparam)
    print(req.status_code)
    soup = BeautifulSoup(req.text, "html.parser")
    for link in soup.find_all("tr", attrs={"class": "listing-list-item"}):
        if link.find("a","listing-text-new")==None:
            continue
        urls.append(link.find("a","listing-text-new").get("href"))
        urlMoney.append(link.find("a","listing-price").text.strip())
        j+=1
        print(str(j))
    sleepTimes = [8.1, 7.5, 9.2,5.3,6.4]
    time.sleep(random.choice(sleepTimes))
dataset=dataFrameMake()
dataset["fiyat"]=urlMoney

200
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
200
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
200
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
200
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
200
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271


In [7]:
#Siteden verileri topluyor ve onları dataframe tablosuna dolduruyor. Ondan sonra her iterasyonda dataframei kaydediyor
for x in range(0,500):
    sleepTimes = [7.1, 8.8, 9.2,6.3,10.4]
    time.sleep(random.choice(sleepTimes))
    req = requests.get(urls[x],headers=headersparam)
    if req.status_code!=200:
        print(str(x),urls[x])
        print("Hata Kodu: "+str(req.status_code))
        continue
    soup2 = BeautifulSoup(req.text, "html.parser")
    
    if (soup2.find("ul","detail-menu")==None):
        print("Hata Kodu: Veri Yok")
        continue
    ozellik=popItem(soup2.find("ul","detail-menu").findAll("span","bold"))
    count=0
    for i in ozellik: 
        dataset.at[x,i]=ozellik[i]
        count=count+1
    if count<12:
        print("eksik veri",str(x),urls[x])
    else:
        print(str(x),urls[x])
    dataset.to_csv('savedCsv.csv',index = False)

0 https://www.arabam.com/ilan/galeriden-satilik-nissan-almera-1-5-tekna/galeriden-nissan-almera-1-5-tekna-2007-model-istanbul/17270410
1 https://www.arabam.com/ilan/galeriden-satilik-nissan-pulsar-1-2-visia/boyasiz-degisensiz-tramersiz-otomatik-vit-pulsar-1-2-dig-t/18622138
2 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-match/west-ten-micra-match-hatasiz-boyasiz-sensor-celik-jant-kredi/18617768
3 https://www.arabam.com/ilan/galeriden-satilik-nissan-primera-1-6-special-edition/galeriden-nissan-primera-1-6-special-edition-1999-model-denizli/18551740
4 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-tekna/yildirim-motors-nissan-micra-1-2-tekna-2005-model-170-000-km/18485116
5 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-3-gx/ozpam-2-el-den-1998-nissan-micra-1-3-gx-n-cvt-benzin-lpg/18544392
6 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-tekna/gold-auto-a-s-stoklarimizda-98-adet-otomatik-nissan-micralar-mevcuttur/15347585
7 

62 https://www.arabam.com/ilan/sahibinden-satilik-nissan-almera-1-5-tekna/sahibinden-bakimli-temiz-nissan-almera-1-5-tekna-2006-model/18614860
63 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-2-0-se/sahibinden-nissan-primera-2-0-se-1998-model/18615337
64 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-tekna/otomatik-micra/18593142
65 https://www.arabam.com/ilan/galeriden-satilik-nissan-almera-1-5-luxury/ates-den-2003-almera-sedan-1-5-luxury-degisensiz-219-000km-temiz/18199958
66 https://www.arabam.com/ilan/galeriden-satilik-nissan-primera-1-6-comfort/aydinlar-otomotiv-den-nadir-temizlikte-nissan-primera-comford/18617734
67 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-match/2017-nissan-micra-1-2-match-otomatik-tekno-mavi-20-o00km-hatasiz/18617674
eksik veri 68 https://www.arabam.com/ilan/sahibinden-satilik-nissan-sunny-1-6-ex/sahibinden-nissan-sunny-1-6-ex-1996-model/13199377
69 https://www.arabam.com/ilan/sahibinden-satilik-nissan-maxima

124 https://www.arabam.com/ilan/sahibinden-satilik-nissan-almera-1-5-tekna/sahibinden-nissan-almera-1-5-tekna-2006-model/18589381
125 https://www.arabam.com/ilan/sahibinden-satilik-nissan-note-1-6-tekna/sahibinden-nissan-note-1-6-tekna-2007-model/18588357
126 https://www.arabam.com/ilan/sahibinden-satilik-nissan-almera-1-5-tekna/sahibinden-nissan-almera-1-5-tekna-2005-model-kocaeli/18588809
127 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-passion/ozsaglam-dan-2013-model-nissan-micra-1-2-passion-91binde-beyaz/18589971
128 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-match/2018-micra-1-2-match-ot-vites-80-hp-6-ileri-ful-orjinal/18589612
129 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-tekna/gold-auto-a-s-stoklarimizda-98-adet-otomatik-micralar-mevcuttur/13580863
130 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-street/2017-model-nissan-micra-1-2-street-otomatik-km-47-000-de/18587703
131 https://www.arabam.com/ilan/gal

eksik veri 185 https://www.arabam.com/ilan/sahibinden-satilik-nissan-maxima-2-0-qx/sahibinden-nissan-maxima-2-0-qx-1998-model/18555187
186 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-passion/saray-otomotiv-den-nissan-micra-1-2-passion-75-000km-lpgli/18557165
187 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-1-6-tekna/acil-acil-fiyat-dustu2006-162-500-de-konfor-araci-primera/16156176
188 https://www.arabam.com/ilan/sahibinden-satilik-nissan-micra-1-2-match/sahibinden-nissan-micra-1-2-match-2017-model/18552563
189 https://www.arabam.com/ilan/galeriden-satilik-nissan-almera-1-6-sr/2000-nissan-almera-1-6-yeni-nesil-motor-klimali/18552976
190 https://www.arabam.com/ilan/galeriden-satilik-nissan-note-1-5-dci-tekna-pack/alanyada-vadeli-takasli-nissan-tekna-diesel/17396025
191 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-1-6-visia/sahibinden-nissan-primera-1-6-visia-2005-model/18548127
192 https://www.arabam.com/ilan/sahibinden-satilik-nissan

248 https://www.arabam.com/ilan/sahibinden-satilik-nissan-almera-1-5-tekna/sahibinden-nissan-almera-1-5-tekna-2004-model/17066388
249 https://www.arabam.com/ilan/sahibinden-satilik-nissan-note-1-4-tekna/memurdan-nissan-note-1-4-tekna-2010-model-lg-li/17289044
250 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-tekna/gold-auto-a-s-stoklarimiz-da-98-adet-otomatik-nissan-micra-mevcuttur/13495778
251 https://www.arabam.com/ilan/sahibinden-satilik-nissan-micra-1-2-tekna/sahibinden-nissan-micra-1-2-tekna-2006-model/18517967
252 https://www.arabam.com/ilan/sahibinden-satilik-nissan-micra-1-5-dci-passion/sahibinden-nissan-micra-1-5-dci-passion-2006-model/18517846
253 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-2-0-elegance/sahibinden-nissan-primera-2-0-elegance-2000-model/18514019
254 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-1-6-special-edition/sahibinden-nissan-primera-1-6-special-edition-1999-model/18090157
255 https://www.arabam.com/ilan

309 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-tekna/yalnizca-kibrista-satiliktir-2005-nissan-march-1-2-benzin-sagdan-direksiyonlu-kibris/15434077
310 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-1-6-comfort/sahibinden-nissan-primera-1-6-comfort-2000-model/18470075
311 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-street/hk-car-dan-2014-nissan-micra-89-000-km-de-tam-otomatik/18469820
312 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-tekna/otomix-2005-nissan-micra-tekna-tam-otomatik-sadece-138000-km/15339138
313 https://www.arabam.com/ilan/sahibinden-satilik-nissan-micra-1-5-dci-passion/sahibinden-nissan-micra-1-5-dci-passion-2007-model/17421519
314 https://www.arabam.com/ilan/sahibinden-satilik-nissan-almera-1-5-tekna/sahibinden-nissan-almera-1-5-tekna-2004-model-usak/18037033
315 https://www.arabam.com/ilan/sahibinden-satilik-nissan-micra-1-2-tekna/sahibinden-nissan-micra-1-2-tekna-2011-model/17643882
316 https://w

371 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-1-6-tekna/sahibinden-nissan-primera-1-6-tekna-2005-model/18411810
372 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-1-6-visia/sahibinden-nissan-primera-1-6-visia-2004-model/17977022
373 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-1-6-visia/sahibinden-nissan-primera-1-6-visia-2003-model/17972153
374 https://www.arabam.com/ilan/sahibinden-satilik-nissan-micra-1-2-passion/sahibinden-nissan-micra-1-2-passion-2010-model/18407562
375 https://www.arabam.com/ilan/sahibinden-satilik-nissan-note-1-6-tekna/sahibinden-nissan-note-1-6-tekna-2006-model/17980137
376 https://www.arabam.com/ilan/galeriden-satilik-nissan-nx-coupe-1-6/kuzey-motors-mersin-den-nissan-nx-coupe-1-6-cabrio-2002-cikisli/17945646
377 https://www.arabam.com/ilan/sahibinden-satilik-nissan-almera-1-5-tekna/sahibinden-nissan-almera-1-5-tekna-2006-model/18407777
378 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-tekna/

434 https://www.arabam.com/ilan/sahibinden-satilik-nissan-micra-1-2-punch/sahibinden-nissan-micra-1-2-punch-2012-model/18342539
435 https://www.arabam.com/ilan/sahibinden-satilik-nissan-note-1-5-dci-tekna/sahibinden-acil-satilik-nissan-qashqai-1-5-dci-qashqai-2012-model/18340646
436 https://www.arabam.com/ilan/sahibinden-satilik-nissan-micra-1-4-tekna/sahibinden-nissan-micra-1-4-tekna-2005-model/18341435
437 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-2-0-se/sahibinden-nissan-primera-2-0-se-1999-model-trabzon/17534653
438 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-1-6-tekna/sahibinden-nissan-primera-1-6-tekna-2004-model/18336565
439 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-2-0-elegance/nissan-primera-2-0-elegance-full-otomatic-vites/17866447
440 https://www.arabam.com/ilan/sahibinden-satilik-nissan-note-1-2-tekna/sahibinden-nissan-note-1-2-tekna-2014-model/17899107
441 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primer

497 https://www.arabam.com/ilan/sahibinden-satilik-nissan-primera-1-6-comfort/sahibinden-nissan-primera-1-6-comfort-2000-model-deri-koltuk-4-cam-otomatik/18230093
498 https://www.arabam.com/ilan/galeriden-satilik-nissan-micra-1-2-street/ismira-auto-danmicra-streetotomatik2015-mdl32-000-km-de/18233327
499 https://www.arabam.com/ilan/galeriden-satilik-nissan-primera-1-6/sifir-vize-lpg-li-nissan-primera-2000-model/12642891


In [8]:
#dataframei bir kez daha kaydediyorum
dataset.to_csv('testCsv.csv',index = False)

In [ ]:
#dataframe ilanların urlerini ekliyorum ve ayrı bir şekilde kaydediyorum
for x in range(len(urls)):
    urls[x]=url+urls[x]
dfs=dataset.copy()
dfs["url"]=urls
dfs.to_csv('urlVeri.csv',index = False)
dfs

,marka,seri,model,yil,yakit tipi,vites tipi,motor hacmi,motor gucu,kilometre,boya-degisen,takasa uygun,kimden,fiyat,url
0,nissan,almera,1.5 tekna,2007,benzin,duz,1498 cc,98 hp,97.000 km,1 degisen,takasa uygun degil,galeriden,115.000 TL,https://www.arabam.comhttps://www.arabam.com/i...
1,nissan,pulsar,1.2 visia,2017,benzin,yari otomatik,1200 cm3' e kadar,101 - 125 hp,53.000 km,belirtilmemis,takasa uygun,galeriden,199.000 TL,https://www.arabam.comhttps://www.arabam.com/i...
2,nissan,micra,1.2 match,2016,benzin,yari otomatik,1200 cm3' e kadar,76 - 100 hp,68.000 km,belirtilmemis,takasa uygun,galeriden,183.000 TL,https://www.arabam.comhttps://www.arabam.com/i...
3,nissan,primera,1.6 special edition,1999,lpg & benzin,duz,1597 cc,100 hp,285.000 km,1 degisen,takasa uygun,galeriden,71.500 TL,https://www.arabam.comhttps://www.arabam.com/i...
4,nissan,micra,1.2 tekna,2005,lpg & benzin,duz,1240 cc,80 hp,170.500 km,tamami boyali,takasa uygun,galeriden,82.500 TL,https://www.arabam.comhttps://www.arabam.com/i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,nissan,micra,1.2 tekna,2005,benzin,otomatik,1201 - 1400 cm3,76 - 100 hp,145.000 km,belirtilmemis,takasa uygun degil,galeriden,117.900 TL,https://www.arabam.comhttps://www.arabam.com/i...
496,nissan,primera,1.6 visia,2005,lpg & benzin,duz,1596 cc,109 hp,317.000 km,belirtilmemis,takasa uygun degil,sahibinden,75.000 TL,https://www.arabam.comhttps://www.arabam.com/i...
497,nissan,primera,1.6 comfort,2000,lpg & benzin,duz,1597 cc,100 hp,316.000 km,belirtilmemis,takasa uygun,sahibinden,73.000 TL,https://www.arabam.comhttps://www.arabam.com/i...
498,nissan,micra,1.2 street,2015,benzin,yari otomatik,1200 cm3' e kadar,76 - 100 hp,32.000 km,belirtilmemis,takasa uygun,galeriden,154.500 TL,https://www.arabam.comhttps://www.arabam.com/i...
